<a href="https://colab.research.google.com/github/annieco29/wid_climate_change_datathon/blob/master/transformers_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:00


NameError: ignored

In [4]:
from transformers import pipeline
import pandas as pd
sentiment_task = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest", tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest")
sentiment_task("Covid cases are increasing fast!")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'label': 'negative', 'score': 0.7235767841339111}]

In [30]:
df = pd.read_csv("bbc_science_and_climate_articles_2010-2023.csv")

In [31]:
df.head()

,Unnamed: 0,title,link,description,published
0,0,Mars rover's wind sensor damaged,http://www.bbc.co.uk/news/science-environment-...,A sensor on the mast of Nasa's Curiosity rover...,"Tue, 21 Aug 2012 20:03:54 GMT"
1,1,Bugs sunbathe to 'stay healthy',http://www.bbc.co.uk/nature/19319086,Western boxelder bugs sunbathe to fight off ge...,"Wed, 22 Aug 2012 08:08:03 GMT"
2,2,Star is caught devouring planet,http://www.bbc.co.uk/news/science-environment-...,Astronomers spot evidence of a distant star co...,"Tue, 21 Aug 2012 15:55:38 GMT"
3,3,Arctic ice set to hit record low,http://www.bbc.co.uk/news/science-environment-...,Arctic sea ice looks set to reach a record low...,"Tue, 21 Aug 2012 10:55:46 GMT"
4,4,Shanghai tops 'flood risk list',http://www.bbc.co.uk/news/science-environment-...,Shanghai is the most vulnerable major city in ...,"Tue, 21 Aug 2012 06:58:06 GMT"


In [32]:
df = df.drop(columns = 'Unnamed: 0', axis = 1)

In [13]:
data_sample = df.head(5).copy()

In [33]:
# Load Twitter-roBERTa-base for Sentiment Analysis
sentiment_task = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest", tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest")

# Function to get sentiment
def get_sentiment_label(text):
    # Process text with roBERTa
    doc = sentiment_task(text)
    doc_dict = doc[0]
    # Return the polarity
    return doc_dict['label']

def get_sentiment_score(text):
    # Process text with roBERTa
    doc = sentiment_task(text)
    doc_dict = doc[0]
    # Return the polarity
    return doc_dict['score']

# Apply the function to the text column and create a new column 'sentiment'
df['roberta_title_label'] = df['title'].apply(lambda x: pd.Series(get_sentiment_label(x)))
df['roberta_title_score'] = df['title'].apply(lambda x: pd.Series(get_sentiment_score(x)))
df['roberta_desc_label'] = df['description'].apply(lambda x: pd.Series(get_sentiment_label(x)))
df['roberta_desc_score'] = df['description'].apply(lambda x: pd.Series(get_sentiment_score(x)))

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd

# Define the functions
def get_sentiment_label_and_score(text):
    label, score = get_sentiment_label(text), get_sentiment_score(text)
    return label, score

# Apply the functions to the 'title' and 'description' columns
result_columns = df[['title', 'description']].applymap(get_sentiment_label_and_score)

# Rename the resulting columns
result_columns.columns = ['roberta_title', 'roberta_description']

# Concatenate the result_columns DataFrame with the original df
df = pd.concat([df, result_columns], axis=1)

# The df DataFrame now contains 'roberta_title' and 'roberta_description' columns
